Last updated 28 August 2017

In [1]:
import pandas as pd
import numpy as np
import scipy.io as sio
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

# Support Vector Machines

In the first half of this exercise, we will be using support vector machines (SVMs) with various example 2D datasets. Experimenting with these datasets will help us gain an intuition of how SVMs work and how to use a Gaussian kernel with SVMs.

## Example Dataset 1

We begin by with a 2D example dataset which can be separated by a linear boundary. Observe that there is an outlier that contradicts with the natural separation implied between the positive and negative examples. We will later observe how this outlier affects the SVM decision boundary.

In [2]:
data1 = sio.loadmat("ex6data1.mat")
X1 = data1["X"] ; y1 = data1["y"].flatten()

# obtain boolean mask
mask0 = y1 == 0
mask1 = y1 ==1 # mask for class 1
cl10 = go.Scatter(x = X1[mask0][:,0],  # boolean indexing out class 0, and column 0 of dataset
                    y= X1[mask0][:,1], mode = "markers", name= "Class 0")
cl11 = go.Scatter(x = X1[mask1][:,0], 
                    y= X1[mask1][:,1], mode = "markers", name= "Class 1")
plotly.offline.iplot({"data": [cl10,cl11],
                      "layout": go.Layout(title = "Example Dataset 1",
                                          xaxis = dict(title="X1"),
                                          yaxis = dict(title="X2"))})

We will not be doing our own implementation of the learning routine for training our SVM. SVM optimization is a much more complicated problem than prior classification problems in this course, since it involves constratined optimization. It will be best to just use a standard package implemented by experts.

Being consistent with the course's usage of an SMO based algorithm for training the SVM classifier, we will use `scikit-learn SVC` even though the alternative, `LinearSVC`, might have been the more convenient classifier to use (since it directly applies a linear model). 

In [3]:
from sklearn.svm import SVC

clf = SVC(C=1, kernel = 'linear') # use C=1 for this part
clf.fit(data1["X"], data1["y"].flatten())

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [4]:
print(clf.predict(data1["X"]))
print(clf.score(data1["X"], data1["y"]))

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
0.980392156863


### Decision boundary is generated without directly using the parameters learned in our SVM model
Generating the plot of the SVM decision boundary in this exercise is not quite the same as how we generated the decision boundary for our logsitic regression classifiers in [exercise 2](https://nbviewer.jupyter.org/github/kohaugustine/Machine-Learning-Coursera/blob/master/machine-learning-ex2/Programming%20Exercise%202%20-%20Logistic%20Regression.ipynb), the reason being that we do not have the ability to retrieve out $\theta$ vector of learned parameters from the trained model, and even if we could, we do not have a simple expression of the hypothesis function which we can use to compute either the class values directly, or the probabilities that an example in the data belongs to a certain class.

The functions associated with the SVM classifier are mathematically much more complicated to solve than that of the logistic regression classifier. The optimization objective of the SVM is actually an example of a quadratic program. A special constrained optimization algorithm, the [SMO](https://en.wikipedia.org/wiki/Sequential_minimal_optimization), is used. It does not actually compute using the parameterized cost function as taught in the lectures, neither does it give us a trained vector of parameters, $\theta$.

Instead, it is more likely that the implementation in LIBSVM uses a data structure called the "model" that represents the factors used in the SMO method, and therefore the model, which is wrapped underneath the scikit-learn interface, is just directly used to make predictions on input data given to it.

Although the decision boundary here is represented as a contour plot, just like how it was in the non-linear logistic regression case in exercise 2, we directly pass the trained model a meshgrid of input data examples encompassing the entire space, and get back the predictions directly which we use to obtain our decision boundary, never needing to access the model's parameters at all.

### Drawing the SVM decision boundary using a mesh grid of input data generated with vectorized code in `generate_mesh_matrix()`

Here we can perform a vectorized version of mesh grid generation to draw the decision boundary, because unlike the [logistic regression for non-linear decision boundary prediction problem in exercise 2](https://nbviewer.jupyter.org/github/kohaugustine/Machine-Learning-Coursera/blob/master/machine-learning-ex2/Programming%20Exercise%202%20-%20Logistic%20Regression.ipynb#Plotting-the-decision-boundary), where we used polynomial features, here our feature space is small (as I am using these features as they are, and not engineering any new ones) so our memory requirements are much smaller and manageable by my workstation.

TODO: improve my implementation by following [SKlearn example](http://scikit-learn.org/stable/auto_examples/svm/plot_iris.html#sphx-glr-auto-examples-svm-plot-iris-py) of using `np.c_[]` or `np.r_[]` to perform concatenation and stacking. What's special about these methods? https://docs.scipy.org/doc/numpy-1.8.1/reference/generated/numpy.c_.html#numpy.c_

In [5]:
def generate_mesh_matrix(_X):
    _x0 = np.linspace(start=_X[:,0].min(), stop = _X[:,0].max(), num=200)
    _x1 = np.linspace(start=_X[:,1].min(), stop = _X[:,1].max(), num=200)
    # Now we generate the meshes along the respective dimensions
    _x0m, _x1m = np.meshgrid(_x0,_x1)

    return np.array(list(zip(_x0m.flatten(), _x1m.flatten()))), _x0,_x1

In [6]:
data_mat, x0,x1 = generate_mesh_matrix(data1["X"])
pred_grid = clf.predict(data_mat).reshape((len(x0),len(x0)))

Remember that the `x` and `y` arguments to `go.Contour` must be 1D arrays. Previously Passing them in as 2D meshes resulted in [cryptic errors](https://community.plot.ly/t/solved-plotting-an-svm-decision-boundary-using-go-contour/5418).

In [7]:
contmap1 = go.Contour(z = pred_grid, x=x0, y=x1, showlegend = True, 
                         name = "Prediction when C = 10", 
                        text =np.asarray(list(map(lambda x: "Class 1" if x == 1 else "Class 0",
                                            pred_grid.flatten()))).reshape((len(x0),len(x0))),
                        hoverinfo = "name+x+y+text" ,
                         ncontours = 5,
                         line = dict(smoothing=1.3, width = 2),
                         contours = dict(coloring="lines"), 
                          showscale = False)

plotly.offline.iplot({
        "data": [cl10,cl11,contmap1],
        "layout" : go.Layout(title = "SVM Decision Boundary with C = 1",
                             xaxis = dict(title="X1"),
                             yaxis = dict(title="X2"))
})

When $C = 1$, we find, as shown above, that the SVM puts the decision boundary in the gap that maximizes the margin between the two classes, and misclassifies the data point on the far left.

In [8]:
clf = SVC(C=100, kernel = 'linear')
clf.fit(data1["X"], data1["y"].flatten())
pred_grid = clf.predict(data_mat).reshape((len(x0),len(x0)))

contmap = go.Contour(z = pred_grid, x=x0, y=x1, showlegend = True, 
                         name = "Prediction", 
                        text =np.asarray(list(map(lambda x: "Class 1" if x == 1 else "Class 0",
                                            pred_grid.flatten()))).reshape((len(x0),len(x0))),
                        hoverinfo = "name+x+y+text" ,
                         ncontours = 5,
                         line = dict(smoothing=1.3, width = 2),
                         contours = dict(coloring="lines"), 
                          showscale = False)

plotly.offline.iplot({
        "data": [cl10,cl11,contmap],
        "layout" : go.Layout(title = "SVM Decision Boundary with C = 100",
                             xaxis = dict(title="X1"),
                             yaxis = dict(title="X2"))
})

When we now try $C = 100$, we find that the SVM now classifies every single example correctly, but has a decision boundary that does not appear to be a natural fit for the data. In other words, the SVM has now **overfitted** to the data. $C$ plays a role similar to $\frac{1}{\lambda}$ , where $\lambda$ is the regularization parameter that we were using previously for logistic regression.

# SVM with Gaussian Kernel

In this part of the exercise, we will be using SVMs to do non-linear classification on datasets that are not linearly separable.

## Gaussian Kernel

The Gaussian Kernel function is defined as:
$$ K_{gaussian}(x^{(i)},x^{(j)})=\exp{\Bigg(-\frac{||x^{(i)}-x^{(j)} ||^2}{2\sigma^2}\Bigg)} = \exp{\Bigg(-\frac{\sum_{k=1}^n\left(x_k^{(i)} - x_k^{(j)}\right)^2}{2\sigma^2}\Bigg)}$$

#### Implementation notes for `build_k_gaussian()` kernel function

The interface to `sklearn.SVC` allows us to pass in the custom kernel only as a function object to the instance of the `SVC` class. In order to support later experiments to pass in varying $\sigma$ values as parameters to the kernel function, it is necessary to employ [function closure](https://datascience.stackexchange.com/questions/22416/passing-a-custom-kernel-with-more-than-two-arguments-into-svm-svc-in-scikit-le).

An initial implementation to compute the euclidean distance with `normsq = np.square(np.linalg.norm(_x1-_x2))` did not work because it used `np.linalg.norm()` which returns the norm as a single scalar floating point object, which caused the kernel function to return the result as a single numerical object that is not supported by `SVC`. Instead, using `sklearn.metrics.pairwise.euclidean_distances()` allows us to return the scalar result (since mathematically, by definition, the squared norm between two vectors is always a scalar) as a 2D numpy array of shape `(1,1)` containing just 1 element, as can be seen from this [example](http://scikit-learn.org/stable/auto_examples/svm/plot_custom_kernel.html#sphx-glr-auto-examples-svm-plot-custom-kernel-py ) and the output in the testcases below. See this [discussion](https://stackoverflow.com/questions/31599624/user-defined-svm-kernel-with-scikit-learn) for the issues that arise when using a custom kernel that returns values as non-arrays.



In [9]:
from sklearn.metrics.pairwise import euclidean_distances

def build_k_gaussian(_sigma):

    def k_gaussian(_x1, _x2):
        normsq = euclidean_distances(_x1,_x2, squared=True) # return the squared distances
        #normsq = np.square(np.linalg.norm(_x1-_x2))
        return np.exp(- normsq/(2 * np.square(_sigma)))
    
    return k_gaussian

#### Test cases for Gaussian Kernel

The original `sklearn.SVC` class accepts each data matrix , `X`, input as being a 2D array with each row representing one training example, and each column representing a feature. Our custom kernel must also respect this convention. In order to pass in the test case arrays correctly, imagine that each testcase array, `x1` and `x2`, represents a specific training example, that was indexed out from some larger data matrix `X`. `x1` and `x2` are row vectors with 3 columns, thus 3 features.

As explained briefly above, `build_k_gaussian()` will accept `x1` and `x2` and return me a single value, which mathematically, is a scalar, but in numpy terms is a 2D array of shape `(1, 1)`.

**This is the reason why we had to pass in `x1` and `x2` as row vectors, not column vectors.** If we passed them in as column vectors, as what the MATLAB code for the course does, its as if our testcase comprised of 3 training examples, with 1 feature each (since there is only 1 col).

In [10]:
# Passes all test cases
x1 = np.array([1,2,1]).reshape(1, -1) ; x2 = np.array([0,4,-1]).reshape(1, -1) ; sigma = 2
print(build_k_gaussian(sigma)(x1,x2))
x1 = np.array([1, 2, 3]).reshape(1, -1); x2 = np.array([2, 4, 6]).reshape(1, -1); sigma = 3
print(build_k_gaussian(sigma)(x1,x2))

[[ 0.32465247]]
[[ 0.45942582]]


## Example Dataset 2

In this part, we apply an SVM to dataset 2. From the visualization below, we observe that there is no linear decision boundary that separates the positive and negative examples for this dataset. However, by using the Gaussian kernel with the SVM, we will be able to learn a non-linear decision boundary that can perform reasonably well for the dataset.

In [11]:
data2 = sio.loadmat("./ex6data2.mat")
X2 = data2["X"] ; y2= data2["y"].flatten()
# obtain boolean mask
mask0 = y2 == 0
mask1 = y2 ==1 # mask for class 1
# dataset 2 class 0
cl20 = go.Scatter(x = X2[mask0][:,0],  # boolean indexing out class 0, and column 0 of dataset
                    y= X2[mask0][:,1],
                    mode = "markers", name= "Class 0")
cl21 = go.Scatter(x = X2[mask1][:,0], 
                    y= X2[mask1][:,1],
                       mode = "markers", name= "Class 1",marker=dict(color="red"))
plotly.offline.iplot({"data": [cl20,cl21],
                      "layout": go.Layout(title = "Example Dataset 2",
                                          xaxis = dict(title="X1"),
                                          yaxis = dict(title="X2"))})

In [12]:
clf = SVC(C=1, kernel = build_k_gaussian(0.1))
clf.fit(X2, y2)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto',
  kernel=<function build_k_gaussian.<locals>.k_gaussian at 0x7fc1713b4950>,
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [13]:
data_mat, x0,x1 = generate_mesh_matrix(X2)
pred_grid = clf.predict(data_mat).reshape((len(x0),len(x0)))

contmap = go.Contour(z = pred_grid, x=x0, y=x1, showlegend = True, 
                         name = "Prediction", 
                        text =np.asarray(list(map(lambda x: "Class 1" if x == 1 else "Class 0",
                                            pred_grid.flatten()))).reshape((len(x0),len(x0))),
                        hoverinfo = "name+x+y+text" ,
                         ncontours = 5,
                         line = dict(smoothing=1.3, width = 2),
                         contours = dict(coloring="lines"), 
                          showscale = False)

plotly.offline.iplot({
        "data": [cl20,cl21,contmap],
        "layout" : go.Layout(title = "Gaussian Kernel SVM Decision Boundary for Example Dataset 2",
                             xaxis = dict(title="X1"),
                             yaxis = dict(title="X2"))
})

We see that the decision boundary is able to separate most of the positive and negative examples correctly and follows the contours of the dataset well.

### Another example of when to use custom kernels

This [article](https://stackoverflow.com/questions/37468368/custom-kernels-for-svm-when-to-apply-them?rq=1) uses a nicely motivated example showing how a custom kernel is necessary when the data is in a graph format, as such a kernel will transform data into suitable form for `sklearn` to process, since the SVM implementations in `sklearn` are unable to process graph data directly.

## Example Dataset 3

In this part of the exercise, we will gain more practical skills on how to use a SVM with a Gaussian kernel with hyperparameter fine-tuning. We first load and display the third dataset.

In [14]:
data3 = sio.loadmat("./ex6data3.mat")
X3 = data3["X"]; y3 = data3["y"].flatten() ; X3val = data3["Xval"]; y3val = data3["yval"].flatten()
# obtain boolean mask
mask0 = y3 == 0
mask1 = y3 ==1 # mask for class 1
maskval0 = y3val == 0
maskval1 = y3val == 1
# dataset 3 training, class 0
cl30 = go.Scatter(x = X3[mask0][:,0],  # boolean indexing out class 0, and column 0 of dataset
                    y= X3[mask0][:,1],
                    mode = "markers", name= "Training Data Class 0", marker=dict(color="blue"))
cl31 = go.Scatter(x = X3[mask1][:,0], 
                    y= X3[mask1][:,1],
                       mode = "markers", name= "Training Data Class 1",marker=dict(color="red"))
cl30val = go.Scatter(x = X3val[maskval0][:,0], 
                    y= X3val[maskval0][:,1],
                       mode = "markers", name= "Cross-Validation Data Class 0",marker=dict(symbol="asterisk-open",
                                                                                   color = 'blue'))
cl31val = go.Scatter(x = X3val[maskval1][:,0], 
                    y= X3val[maskval1][:,1],
                       mode = "markers", name= "Cross-Validation Data Class 1",marker=dict(symbol= "asterisk-open",
                                                                                   color="red"))

plotly.offline.iplot({"data": [cl30,cl31, cl30val, cl31val],
                      "layout": go.Layout(title = "Example Dataset 3",
                                          xaxis = dict(title="X1"),
                                          yaxis = dict(title="X2"))})

The task is to use the cross validation set `Xval`, `yval` to determine the best $C$ and $\sigma$ hyperparameters to use. It is suggested that for both $C$ and $\sigma$, try their values in multiplicative steps (e.g., 0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30). We will try all possible pairs of values for C and (e.g., $C$ = 0.3 and $\sigma$ = 0.1). Validation curves will be our tool of choice to help us decide these hyperparameters.

### Validation Curves

Our approach to this is to plot a figure with $C$ on the x-axis, and accuracy score on the y-axis. For each chosen value of $\sigma$, we generate a validation curve that stretches horizontally along the figure, showing the model accuracy for each value of $C$.

In [15]:
# Train model on training data using all 84 combinations of C and sigma, and then score our model on both training
# and cross val data for each combo of C,sigma
Cs = [0.01,0.03,0.1,0.3,1,3,10,30]; sigmas= [0.01,0.03,0.1,0.3,1,3,10,30]
errors = np.zeros((2,len(sigmas),len(Cs))) # first axis has only 2 dims, first dim stores training errors, second dim stores cross val errors
for i in range(0,len(sigmas)):
    for j in range(0,len(Cs)):
        clf = SVC(C=Cs[j], kernel = build_k_gaussian(sigmas[i]))
        clf.fit(X3, y3)
        errtrg = clf.score(X3, y3) # assess scoring accuracyt
        errval = clf.score(X3val, y3val) # assess scoring accuracy on the validation set
        errors[0][i][j] = errtrg
        errors[1][i][j] = errval
# visualize the results of the hyperparameter search as validation curves
curves = []
colors = ["blue","green", "cyan","orange","red", "brown", "black", "pink"]
for t in [0,1]:
    row = 0 # index out the correct set of error values
    for sigma in sigmas:
        l = go.Scatter(x = Cs, 
                       y= errors[t][row],
                       mode = "line", 
                       line = dict(dash="solid" if t==0 else "dot", color = colors[row]),
                       name= "{}, Sigma {}".format("Training" if t==0 else "Cross-Validation", sigma)) 
                        #,marker=
        row = row + 1
        curves.append(l)
        
plotly.offline.iplot({"data": curves,
                      "layout": go.Layout(title = "Validation Curves for Gaussian Kernel SVM for different combinations of C and sigma",
                                          xaxis = dict(title="Regularization Parameter, C"),
                                          yaxis = dict(title="Accuracy"))})

### Reviewing the results, and choosing the best hyperparameters

The above validation curves are plotted based on accuracy, which increases is the fraction of examples that the SVM predictor classified __correctly__. Therefore, accuracy is a metric that increases as the classifier performance improves. On the other hand, error is fraction of examples that were classfied __incorrectly__. Thus, error is a metric that decreases as classifier performance improves. The [`score()` helper method](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC.score) of the `sklearn.svm.SVC` class gives the accuracy metric. To obtain error, we can use the relationship between accuracy and error as $\text{error} = 1- \text{accuracy}$. 

To remove some curves from the figure to facilitate easier inspection, just click on their names in the legend on the right. Reviewing the plot, we notice that the cyan and orange curves, corresponding to values of $\sigma=0.1,0.3$, gave the highest accuracy scores in the ranges of $C > 0.03$, both on the training and cross-validation datasets. While any combination of the hyperparameters within these ranges should work well, selecting $C=1, \sigma=0.1$, corresponding to the point on the cyan curve with accuracy of $0.965$, gives an SVM decision boundary most similar to Figure 7 of the exercise PDF. This decision boundary is visualized below.

In [16]:
C = 1; sigma = 0.1 # chosen "best" hyperparameters
# Train and fit 
clf = SVC(C=C, kernel = build_k_gaussian(sigma))
clf.fit(X3, y3)
data_mat, x0,x1 = generate_mesh_matrix(X3) # generate input mesh
# prediction on the mesh
pred_grid = clf.predict(data_mat).reshape((len(x0),len(x0)))
#plot decision boundary
contmap = go.Contour(z = pred_grid, x=x0, y=x1, showlegend = True, 
                         name = "Prediction", 
                        text =np.asarray(list(map(lambda x: "Class 1" if x == 1 else "Class 0",
                                            pred_grid.flatten()))).reshape((len(x0),len(x0))),
                        hoverinfo = "name+x+y+text" ,
                         ncontours = 5,
                         line = dict(smoothing=1.3, width = 2),
                         contours = dict(coloring="lines"), 
                          showscale = False)
plotly.offline.iplot({
        "data": [cl30,cl31,contmap],
        "layout" : go.Layout(title = "Gaussian Kernel SVM Decision Boundary using best hyperparameters for Example Dataset 3",
                             xaxis = dict(title="X1"),
                             yaxis = dict(title="X2"))
})

### Further Readings

The SVM is a powerful classifier that finds applications in a myriad of domains, even in food science, where it can be used to [analyze sensory data and support predictive models of consumer preferences](http://www.sciencedirect.com/science/article/pii/S0924224406002433).

### Warning WIP here - using SK-learn pipelines and methods for automated hyperparameter searching

TODO: how do we accomplish this exercise using sklearn already built in functions that facilitate easy search for best hyperparameters? Main issue is when trying to pass in the already segmented out cross-validation datasets....there is no support for doing so....

In [17]:
Cs = [0.01,0.03,0.1,0.3,1,3,10,30]; sigmas= [0.01,0.03,0.1,0.3,1,3,10,30]
from sklearn.model_selection import GridSearchCV
gs = GridSearchCV(SVC(C=1, kernel = build_k_gaussian(0.1)))


TypeError: __init__() missing 1 required positional argument: 'param_grid'

### Warning WIP below - self-exploration work to be completed
TODO: Refer to this reference for list of all availble kernels in SK-learn http://scikit-learn.org/stable/modules/metrics.html

TODO: Is it possible to obtain the feature matrix, after implementing the gaussian kernel, just do an exercise to compute...? It should look something like the self-similarity matrix https://en.wikipedia.org/wiki/Self-similarity_matrix.

Add in this educational segment in following steps:

1. The gaussian kernel follows the conditions as stated in SK-learn metrics documentation, the symmetric, positive definiteness etc
2. Given that the feature matrix is obtained from individual training data examples (insert a Latex matrix) that I drew, and define the individual row vectors
3. do an implementation to produce the feature matrix by arranging the data, and applying kernel function to generate a matrix (with double loop, later vectoriz)
4. As a result of its properties, when we generate the feature map matrix (in step 3), we get positive semidefinite symmetric (whatever, quote wikipedia) which looks like the self similarity matrix or distance matrix (do a heatmap matrix visualization of the feature map matrix showing the symmetry)  https://en.wikipedia.org/wiki/Distance_matrix https://en.wikipedia.org/wiki/Self-similarity_matrix
5. Plot a suface map, to actually show how this symmetry is evident in the nice cone shape of the surface (of course using artificially generated data points), making it look same from all rotational angles - use subplots https://plot.ly/python/subplots/ to plot the heatmap side by side with the surface plot
6. This kernel is said to satisfy mercer's condition, and the feature matrix it produces from the input data satisfies mercer's theorem, which allows us to undertake shortcuts.. (insert explanation from my handwritten notes) - has this got anything to do with diagonalizability?


TODO: Maybe good for further reading section, but to better understand how SVM actually does its work for a non-linear decision boundary, this handwavy explanation helps - if data is not linearly separable, the kernelized SVM (not the basic linear SVM) will be capable of implicitly projecting the data to a higher dimensional space where there exists a hyperplane that can linearly separate the data classes, and once that hyperplane is obtained, drawing a non-linear decision boundary in the original dataspace becomes an easy job... 
https://stats.stackexchange.com/questions/3947/help-me-understand-support-vector-machines/3954#3954
https://stats.stackexchange.com/questions/8769/linear-behaviour-of-nonlinear-svm-in-higher-dimensional-space?rq=1
https://stats.stackexchange.com/questions/69759/feature-map-for-the-gaussian-kernel?noredirect=1&lq=1
https://stats.stackexchange.com/questions/168051/kernel-svm-i-want-an-intuitive-understanding-of-mapping-to-a-higher-dimensional
so the purpose of the feature mapping in SVMs as accomplished through use of a non-linear kernel, is actually quite similar to that in linear or logistic regression, where we also done polynomial feature mapping, however the big difference is that the SVM does not need that mapping to be calculated explicitly as opposed to in the other methods, thereby being much less computationally costly, so that for problems with a huge feature space, the SVM handles it better?

# Spam Classification

TODO: complete this exercise, possibly using sklearn relevant module http://scikit-learn.org/stable/auto_examples/index.html#working-with-text-documents